In [2]:
import sqlite3


In [3]:
# Create connection
conn = sqlite3.connect('school.db')
cursor = conn.cursor()

In [6]:
# Create tables
cursor.execute('''CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    age INTEGER,
    gender TEXT
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS courses (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    credits INTEGER
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS enrollments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id INTEGER,
    course_id INTEGER,
    grade REAL,
    FOREIGN KEY(student_id) REFERENCES students(id),
    FOREIGN KEY(course_id) REFERENCES courses(id)
)''')


In [8]:
# Insert sample data
cursor.executemany('INSERT INTO students (name, age, gender) VALUES (?, ?, ?)', [
    ('Alice', 22, 'F'),
    ('Bob', 20, 'M'),
    ('Charlie', 23, 'M')
])

cursor.executemany('INSERT INTO courses (name, credits) VALUES (?, ?)', [
    ('Math', 3),
    ('Science', 4),
    ('History', 2)
])

cursor.executemany('INSERT INTO enrollments (student_id, course_id, grade) VALUES (?, ?, ?)', [
    (1, 1, 85),
    (1, 2, 90),
    (2, 1, 78),
    (3, 3, 88)
]) 

conn.commit()


In [9]:
# Select all students
cursor.execute("SELECT * FROM students")

#Fetch All Data 
print(cursor.fetchall())


[(1, 'Alice', 22, 'F'), (2, 'Bob', 20, 'M'), (3, 'Charlie', 23, 'M'), (4, 'Alice', 22, 'F'), (5, 'Bob', 20, 'M'), (6, 'Charlie', 23, 'M')]


In [10]:
# Insert a new student
cursor.execute("INSERT INTO students (name, age, gender) VALUES ('David', 21, 'M')")
conn.commit()



In [ ]:
# Filter students older than 21
cursor.execute("SELECT * FROM students WHERE age > 21")
print(cursor.fetchall())

[(1, 'Alice', 22, 'F'), (2, 'Bob', 20, 'M'), (3, 'Charlie', 23, 'M'), (4, 'Alice', 22, 'F'), (5, 'Bob', 20, 'M'), (6, 'Charlie', 23, 'M'), (7, 'David', 21, 'M')]


## joins 
# INNER JOIN 
<img src="https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.reddit.com%2Fr%2Fprogramming%2Fcomments%2F1xlqeu%2Fsql_joins_explained_xpost_rsql%2F&psig=AOvVaw3cYEni5x6GQ1BNrVlO287i&ust=1733509609462000&source=images&cd=vfe&opi=89978449&ved=2ahUKEwi79ZTVoJGKAxUFTEEAHR4hL-IQjRx6BAgAEBk" alt="SQL Joins Explained" width="600">


In [15]:
# Retrieve students with grades between 80 and 90
cursor.execute('''SELECT s.name, e.grade 
                  FROM students s 
                  JOIN enrollments e ON s.id = e.student_id 
                  WHERE e.grade BETWEEN 80 AND 90''')
print(cursor.fetchall())

[('Alice', 85.0), ('Alice', 90.0), ('Charlie', 88.0), ('Alice', 85.0), ('Alice', 90.0), ('Charlie', 88.0)]


In [14]:
# Total credits for each course
cursor.execute("SELECT name, credits FROM courses")
print(cursor.fetchall())



[('Math', 3), ('Science', 4), ('History', 2), ('Math', 3), ('Science', 4), ('History', 2)]


In [13]:
# Students sorted by age descending
cursor.execute("SELECT * FROM students ORDER BY age DESC")
print(cursor.fetchall())

[(3, 'Charlie', 23, 'M'), (6, 'Charlie', 23, 'M'), (1, 'Alice', 22, 'F'), (4, 'Alice', 22, 'F'), (7, 'David', 21, 'M'), (2, 'Bob', 20, 'M'), (5, 'Bob', 20, 'M')]


In [16]:
# Students not enrolled in any courses (LEFT JOIN)
cursor.execute('''SELECT s.name 
                  FROM students s 
                  LEFT JOIN enrollments e ON s.id = e.student_id 
                  WHERE e.id IS NULL''')
print(cursor.fetchall())

[('Alice',), ('Bob',), ('Charlie',), ('David',)]


In [33]:
# Inner join: students and their courses
cursor.execute('''select s.name as student_name , c.name as course_name
             from students s 
            join enrollments e on s.id = e.student_id
            join courses c on c.id = e.course_id''')

#cursor.execute('''select s.name  , c.name 
 #            from students s 
  #           join courses c on s.id = c.id''')


print(cursor.fetchall())

[('Alice', 'Math'), ('Alice', 'Science'), ('Bob', 'Math'), ('Charlie', 'History'), ('Alice', 'Math'), ('Alice', 'Science'), ('Bob', 'Math'), ('Charlie', 'History')]


In [35]:
# Average grade for each course
cursor.execute(''' select c.name , AVG (e.grade) AS average_grade 
               from enrollments e 
               join courses c on c.id = e.course_id 
               group by c.id ''')

print(cursor.fetchall())
# Courses with more than one enrollment
cursor.execute('''select c.name , COUNT(e.id) AS enrollments
               from courses c 
               join enrollments e ON c.id = e.course_id 
               group by c.id having COUNT(e.id) > 1''')
print(cursor.fetchall())


[('Math', 81.5), ('Science', 90.0), ('History', 88.0)]
[('Math', 4), ('Science', 2), ('History', 2)]


## Hands-On Activity
# Objective: Consolidate knowledge by solving problems.
# Task:
- Write a query to find students enrolled in "Math" with grades above 80.
- List all students and their total number of enrolled courses.

In [22]:
# Query to find students enrolled in "Math" with grades above 80
cursor.execute('''SELECT s.name AS student_name, e.grade 
                  FROM students s
                  JOIN enrollments e ON s.id = e.student_id
                  JOIN courses c ON c.id = e.course_id
                  WHERE c.name = 'Math' AND e.grade > 80''')
print(cursor.fetchall())



[('Alice', 85.0)]


In [23]:
# Query to list students and their total number of enrolled courses
cursor.execute('''SELECT s.name AS student_name, COUNT(e.course_id) AS total_courses
                  FROM students s
                  LEFT JOIN enrollments e ON s.id = e.student_id
                  GROUP BY s.id''')
print(cursor.fetchall())

[('Alice', 2), ('Bob', 1), ('Charlie', 1), ('David', 0), ('David', 0), ('David', 0)]


In [24]:
conn.close()

In [ ]:
!pip install pymongo

In [25]:
from pymongo import MongoClient


In [26]:
client = MongoClient("mongodb://localhost:27017/")
db = client["TaskManagement"]
tasks_collection = db["Tasks"]


In [27]:
all_tasks = tasks_collection.find()
for task in all_tasks:
    print(task)

{'_id': ObjectId('6751a0fa43e703d665ad4fee'), 'task_id': 1, 'title': 'Prepare Presentation', 'description': 'Prepare the project presentation for the client.', 'status': 'In Progress', 'priority': 'High', 'due_date': '2024-12-15', 'assigned_to': 'Mohamed'}
{'_id': ObjectId('6751a0fa43e703d665ad4fef'), 'task_id': 2, 'title': 'Finalize Budget', 'description': 'Complete the budget for the new project.', 'status': 'Pending', 'priority': 'Medium', 'due_date': '2024-12-20', 'assigned_to': 'Fatima'}
{'_id': ObjectId('6751a0fa43e703d665ad4ff0'), 'task_id': 3, 'title': 'Team Meeting', 'description': 'Weekly team meeting to discuss project status.', 'status': 'Completed', 'priority': 'Low', 'due_date': '2024-12-05', 'assigned_to': 'Ali'}


In [28]:
in_progress_tasks = tasks_collection.find({"status": "In Progress"})
for task in in_progress_tasks:
    print(task)

{'_id': ObjectId('6751a0fa43e703d665ad4fee'), 'task_id': 1, 'title': 'Prepare Presentation', 'description': 'Prepare the project presentation for the client.', 'status': 'In Progress', 'priority': 'High', 'due_date': '2024-12-15', 'assigned_to': 'Mohamed'}


In [29]:
import pandas as pd


In [36]:
all_tasks = tasks_collection.find()
tasks_list = list(all_tasks)


In [37]:
tasks_df = pd.DataFrame(tasks_list)


In [38]:
tasks_df

,_id,task_id,title,description,status,priority,due_date,assigned_to
0,6751a0fa43e703d665ad4fee,1,Prepare Presentation,Prepare the project presentation for the client.,In Progress,High,2024-12-15,Mohamed
1,6751a0fa43e703d665ad4fef,2,Finalize Budget,Complete the budget for the new project.,Pending,Medium,2024-12-20,Fatima
2,6751a0fa43e703d665ad4ff0,3,Team Meeting,Weekly team meeting to discuss project status.,Completed,Low,2024-12-05,Ali
3,675205b9344e856443e451ce,4,Prepare Presentation,Prepare the project presentation for the client.,In Progress,High,2024-12-15,Mohamed
4,675205b9344e856443e451cf,5,Finalize Budget,Complete the budget for the new project.,Pending,Medium,2024-12-20,Fatima
5,675205b9344e856443e451d0,6,Team Meeting,Weekly team meeting to discuss project status.,Completed,Low,2024-12-05,Ali


In [34]:
tasks = [
    {
        "task_id": 4,
        "title": "Prepare Presentation",
        "description": "Prepare the project presentation for the client.",
        "status": "In Progress",
        "priority": "High",
        "due_date": "2024-12-15",
        "assigned_to": "Mohamed"
    },
    {
        "task_id": 5,
        "title": "Finalize Budget",
        "description": "Complete the budget for the new project.",
        "status": "Pending",
        "priority": "Medium",
        "due_date": "2024-12-20",
        "assigned_to": "Fatima"
    },
    {
        "task_id": 6,
        "title": "Team Meeting",
        "description": "Weekly team meeting to discuss project status.",
        "status": "Completed",
        "priority": "Low",
        "due_date": "2024-12-05",
        "assigned_to": "Ali"
    }
]


In [35]:
tasks_collection.insert_many(tasks)

InsertManyResult([ObjectId('675205b9344e856443e451ce'), ObjectId('675205b9344e856443e451cf'), ObjectId('675205b9344e856443e451d0')], acknowledged=True)